In [2]:
import torch
import transformers
model_name_or_path = 'checkpoints/tinyllama_caption2instruct_ft'
model = transformers.LlamaForCausalLM.from_pretrained(
        model_name_or_path,
        attn_implementation='flash_attention_2',
        torch_dtype=torch.bfloat16,
    ).cuda()
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
from instruct_tri2tri.tinyllama_ft.conversation import conv_llama_2
conv = conv_llama_2.copy()
roles = conv_llama_2.roles
inp = '3D rendering of a white sofa with wooden legs and frame.'
conv.append_message(conv_llama_2.roles[0], inp)
conv.append_message(conv_llama_2.roles[1], None)
prompt = conv.get_prompt()
inputs = tokenizer(prompt, return_tensors='pt')
for key, value in inputs.items():
    inputs[key] = value.cuda()

In [2]:
import json

j = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_550k.json'))

In [3]:
j[0]

{'id': '53d0b31aa7f84bc4b1733224963d0114',
 'image': 'objaverse/Cap3D_imgs_view0/53d0b31aa7f84bc4b1733224963d0114',
 'conversations': [{'from': 'human',
   'value': '<triplane>\nDescribe the 3D model concisely.'},
  {'from': 'gpt',
   'value': '3D rendering of a white sofa with wooden legs and frame.'}]}

In [18]:
output = model.generate(**inputs)
# tokenizer.decode(output[0], special_tokens=True)

In [24]:
tokenizer.decode(output[0, inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

'make the sofa a brown leather sofa'

In [5]:
import json 
json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k_0_4.json'))

[{'image': 'objaverse/Cap3D_imgs_view0/53d0b31aa7f84bc4b1733224963d0114',
  'caption': '3D rendering of a white sofa with wooden legs and frame.',
  'instruct': 'make the sofa a leather sofa'},
 {'image': 'objaverse/Cap3D_imgs_view0/25f25e35aada40e49194657fd51c1199',
  'caption': '3D model of a wooden fence with posts and gate, featuring a sand base.',
  'instruct': 'make the fence made of metal'},
 {'image': 'objaverse/Cap3D_imgs_view0/9ff331513cea44a2938d09a03d7b0493',
  'caption': 'White plastic cylinder resembling a toilet paper holder.',
  'instruct': 'make it a toilet paper holder made of gold'},
 {'image': 'objaverse/Cap3D_imgs_view0/f78e38ccea1d45dcaefe1970e7df4035',
  'caption': '3D model of a forklift truck with a trailer.',
  'instruct': 'make the truck a tank'},
 {'image': 'objaverse/Cap3D_imgs_view0/112c059282cf4511a01fd27211edcae8',
  'caption': 'A 3D model of a wooden table and bench.',
  'instruct': 'make it a wooden table and bench made of marble'},
 {'image': 'objaver